In [1]:
import os
from os import path
from argparse import ArgumentParser

import numpy as np
import torch
import torchvision.transforms.functional as TF
import cv2

from model.network import deeplabv3plus_resnet50 as S2M
from model.aggregate import aggregate_wbg_channel as aggregate
from dataset.range_transform import im_normalization
from util.tensor_util import pad_divide_by
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns
from torchsummary import summary
from torch import nn
from torch.nn import functional as F
import math
from model_base import Base
from typing import List
import cv2
torch.backends.cudnn.benchmark = True

In [2]:
net = Base("resnet50", 7, 1)
net.load_state_dict(torch.load('./saves_2022/0.030462391674518585')['model_state_dict'])
net = net.cuda().eval()
torch.set_grad_enabled(False)

In [3]:
def limit_longest_size(image, max_side_size):
    if image.shape[0] > image.shape[1]:
        target_h = min(image.shape[0], max_side_size)
        target_w = int(image.shape[1] / image.shape[0] * target_h + 0.5)
    else:
        target_w = min(image.shape[1], max_side_size)
        target_h = int(image.shape[0] / image.shape[1] * target_w + 0.5)
    image = cv2.resize(image, (target_w, target_h), interpolation=cv2.INTER_LINEAR)
    return image

In [4]:
from clicker import Clicker
import tqdm

In [5]:
alpha_copy = r'C:\Users\andry\Desktop\Combined_Dataset\Test_set\Composition-1k-testset\alpha_copy'
merged = r'C:\Users\andry\Desktop\Combined_Dataset\Test_set\Composition-1k-testset\merged'
#trimaps = r'C:\Users\andry\Desktop\Combined_Dataset\Test_set\Composition-1k-testset\trimaps'

In [6]:
class InteractiveManager:
    def __init__(self, model, image, mask):
        self.model = model

        self.image = im_normalization(TF.to_tensor(image)).unsqueeze(0).cuda()
        self.mask = TF.to_tensor(mask).unsqueeze(0).cuda()

        h, w = self.image.shape[-2:]
        self.max_size = max(h,w)
        self.image, self.pad = pad_divide_by(self.image, 16)
        self.mask, _ = pad_divide_by(self.mask, 16)
        self.last_mask = None

        # Positive and negative scribbles
        self.p_srb = np.zeros((h, w), dtype=np.uint8)
        self.n_srb = np.zeros((h, w), dtype=np.uint8)
        self.s_srb = np.zeros((h, w), dtype=np.uint8)

        # Used for drawing
        self.pressed = False
        self.last_ex = self.last_ey = None
        self.mode = 0
        self.need_update = True

    def mouse_down(self, ex, ey):
        self.last_ex = ex
        self.last_ey = ey
        self.pressed = True
        if self.mode == 0:
            cv2.circle(self.p_srb, (ex, ey), radius=int(self.max_size * (5/512)), color=(1), thickness=-1)
        elif self.mode == 1:
            cv2.circle(self.n_srb, (ex, ey), radius=int(self.max_size * (5/512)), color=(1), thickness=-1)
        elif self.mode == 2:
            cv2.circle(self.s_srb, (ex, ey), radius=int(self.max_size * (5/512)), color=(1), thickness=-1)
        self.need_update = True

    def mouse_up(self):
        self.pressed = False

    first = True

    def run_s2m(self):
        # Convert scribbles to tensors
        Rsp = torch.from_numpy(self.p_srb).unsqueeze(0).unsqueeze(0).float().cuda()
        Rsn = torch.from_numpy(self.n_srb).unsqueeze(0).unsqueeze(0).float().cuda()
        Rss = torch.from_numpy(self.s_srb).unsqueeze(0).unsqueeze(0).float().cuda()
        Rs = torch.cat([Rsn, Rsp, Rss], 1)
        Rs, _ = pad_divide_by(Rs, 16)

        # Use the network to do stuff
        inputs = torch.cat([self.image, self.mask, Rs], 1)
        mask = net(inputs)
        #inputs = torch.cat([self.image, mask, Rs], 1)
        #mask = net(inputs)
        #print(mask)
        # We don't overwrite current mask until commit            
        self.last_mask = mask
        np_mask = (mask.detach().cpu().numpy()[0,0] * 255).astype(np.uint8)

        if self.pad[2]+self.pad[3] > 0:
            np_mask = np_mask[self.pad[2]:-self.pad[3],:]
        if self.pad[0]+self.pad[1] > 0:
            np_mask = np_mask[:,self.pad[0]:-self.pad[1]]

        return np_mask

    def commit(self):
        self.p_srb.fill(0)
        self.n_srb.fill(0)
        self.s_srb.fill(0)
        if self.last_mask is not None:
            self.mask = self.last_mask

    def clean_up(self):
        self.p_srb.fill(0)
        self.n_srb.fill(0)
        self.s_srb.fill(0)
        self.mask.zero_()
        self.last_mask = None

In [7]:
def comp_image(image, mask, p_srb, n_srb, s_srb):
    color_mask = np.zeros_like(image, dtype=np.uint8)
    color_mask[:,:,2] = 1
    if len(mask.shape) == 2:
        mask = mask[:,:,None]
    comp = (image*0.5 + color_mask*mask*0.5).astype(np.uint8)
    comp[p_srb>0.5, :] = np.array([0, 255, 0], dtype=np.uint8)
    comp[s_srb>0.5, :] = np.array([255, 0, 0], dtype=np.uint8)
    comp[n_srb>0.5, :] = np.array([0, 0, 255], dtype=np.uint8)
    return comp

In [8]:
for name in tqdm.tqdm(os.listdir(merged)):
    gt_alpha = cv2.imread(os.path.join(alpha_copy, name), cv2.IMREAD_GRAYSCALE)
    gt_size = gt_alpha.shape
    gt_alpha = limit_longest_size(gt_alpha, 1024)
    
    image = cv2.imread(os.path.join(merged, name), cv2.IMREAD_COLOR)
    image = limit_longest_size(image, 1024)
    prev_mask = np.zeros_like(gt_alpha)
    
    clicker_pos = Clicker(gt_alpha > 200)
    clicker_neg = Clicker(gt_alpha == 0)
    clicker_trans = Clicker((gt_alpha > 0) & (gt_alpha <= 200))
    
    interactive_manager = InteractiveManager(net, image, prev_mask)
    total_clicks = 0
    for click_indx in range(0, 25):
        
        prev_mask_pos = prev_mask >= 240
        prev_mask_neg = prev_mask <= 15
        prev_mask_trans = (prev_mask > 15) & (prev_mask < 240)
        
        clicker_pos.make_next_click(prev_mask_pos)
        clicker_neg.make_next_click(prev_mask_neg)
        clicker_trans.make_next_click(prev_mask_trans)
        
        clicks_pos = clicker_pos.get_clicks()
        last_click_pos = clicks_pos[-1]
        if last_click_pos.is_positive:
            interactive_manager.mode = 0
            interactive_manager.mouse_down(last_click_pos.coords[1], last_click_pos.coords[0])
#         else:
#             interactive_manager.mode = 1
#             interactive_manager.mouse_down(last_click_pos.coords[1], last_click_pos.coords[0])
            total_clicks += 1
    
        clicks_neg = clicker_neg.get_clicks()
        last_click_neg = clicks_neg[-1]
        if last_click_neg.is_positive:
            interactive_manager.mode = 1
            interactive_manager.mouse_down(last_click_neg.coords[1], last_click_neg.coords[0])
            total_clicks += 1
#         else:
#             interactive_manager.mode = 0
#             interactive_manager.mouse_down(last_click_neg.coords[1], last_click_neg.coords[0])
        
        clicks_trans = clicker_trans.get_clicks()
        last_click_trans = clicks_trans[-1]
        if last_click_trans.is_positive:
            interactive_manager.mode = 2
            interactive_manager.mouse_down(last_click_trans.coords[1], last_click_trans.coords[0])
            total_clicks += 1
#         else:
#             interactive_manager.mode = 1
#             interactive_manager.mouse_down(last_click_trans.coords[1], last_click_trans.coords[0])
            
        prev_mask = interactive_manager.run_s2m()
        
        display = comp_image(image, prev_mask, interactive_manager.p_srb, interactive_manager.n_srb, interactive_manager.s_srb)
        
        os.makedirs(os.path.join('./compositions_vis', name), exist_ok=True)
        cv2.imwrite(os.path.join('./compositions_vis', name, str(click_indx).zfill(3) + '_' + str(total_clicks) + '.jpg'), display) 
        
        outs = cv2.resize(prev_mask, (gt_size[1], gt_size[0]), interpolation=cv2.INTER_LINEAR)
        os.makedirs(os.path.join('./compositions_outputs', name), exist_ok=True)
        #cv2.imwrite(os.path.join('./compositions_outputs', name, str(click_indx).zfill(3) + '_' + str(total_clicks) + '.png'), outs)
        cv2.imwrite(os.path.join('./compositions_outputs', name, str(click_indx).zfill(3) + '.png'), outs) 

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:31:36<00:00,  5.50s/it]


In [12]:
!python evaluation.py

Unknown Region: MSE: 0.08157783694054736  SAD: 289.82121875
Unknown Region: MSE: 0.08063014972383378  SAD: 287.5969375
Unknown Region: MSE: 0.09969395779496792  SAD: 320.7695625
Unknown Region: MSE: 0.09349741585000294  SAD: 301.96128125
Unknown Region: MSE: 0.0835840555674803  SAD: 279.22890625
Unknown Region: MSE: 0.07874134789635265  SAD: 268.77496875
Unknown Region: MSE: 0.07760629303476567  SAD: 265.80740625
Unknown Region: MSE: 0.07731400234678734  SAD: 266.0554375
Unknown Region: MSE: 0.07727627597566046  SAD: 265.2301875
Unknown Region: MSE: 0.07310583817689396  SAD: 256.305046875
Unknown Region: MSE: 0.059478820083716064  SAD: 221.715375
Unknown Region: MSE: 0.060663830620525744  SAD: 222.957171875
Unknown Region: MSE: 0.05664646552471408  SAD: 215.48946875
Unknown Region: MSE: 0.05934440738959521  SAD: 217.4998125
Unknown Region: MSE: 0.05639042414528986  SAD: 210.62153125
Unknown Region: MSE: 0.06034141172609611  SAD: 221.25353125
Unknown Region: MSE: 0.062852972804504  SAD:

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc


Unknown Region: MSE: 0.05168670817206918  SAD: 142.743875
Unknown Region: MSE: 0.05168687152491774  SAD: 142.743953125
Unknown Region: MSE: 0.052469118310667064  SAD: 143.301875
Unknown Region: MSE: 0.05246923499127318  SAD: 143.30184375
Unknown Region: MSE: 0.05246286423017958  SAD: 143.29778125
Unknown Region: MSE: 0.052461847442040625  SAD: 143.29734375
Unknown Region: MSE: 0.05243876468442052  SAD: 143.281484375
Unknown Region: MSE: 0.052726812429857835  SAD: 143.767921875
Unknown Region: MSE: 0.0527208183804354  SAD: 143.76546875
Unknown Region: MSE: 0.05271908817373337  SAD: 143.7653125
Unknown Region: MSE: 0.05271590446005238  SAD: 143.7628125
Unknown Region: MSE: 0.05271376420436316  SAD: 143.761515625
Unknown Region: MSE: 0.0527134808371769  SAD: 143.761171875
Unknown Region: MSE: 0.05271867145728298  SAD: 143.76546875
Unknown Region: MSE: 0.052715294387169  SAD: 143.76509375
Unknown Region: MSE: 0.05293243032767603  SAD: 143.846734375
Unknown Region: MSE: 0.05293243032767603

In [13]:
mse = np.load('./mse.npy', allow_pickle=True)
sad = np.load('./sad.npy', allow_pickle=True)

In [15]:
mse.mean(axis=0)

array([0.09248975, 0.07313974, 0.06161333, 0.05404585, 0.05000278,
       0.04746595, 0.04525883, 0.0437542 , 0.04285818, 0.0421531 ,
       0.04184124, 0.04124405, 0.0410909 , 0.04083048, 0.04075405,
       0.04059643, 0.04049612, 0.04042834, 0.04022895, 0.04013318,
       0.03995988, 0.03998221, 0.03988907, 0.03983318, 0.03977887])

In [19]:
sad.mean(axis=0)

array([105.30840888,  97.10336057,  91.84331159,  87.45200424,
        84.87433358,  83.28603188,  82.13805473,  81.0708592 ,
        80.33062496,  79.86832897,  79.48766048,  79.06590389,
        79.01405008,  78.80780678,  78.73751422,  78.63752608,
        78.54083496,  78.42142316,  78.31301797,  78.22520065,
        78.06181658,  78.05735677,  77.91165147,  77.94507005,
        77.95229326])